In [73]:
import numpy as np

In [74]:
from sklearn.datasets import load_digits

In [75]:
digits = load_digits()
#check the shape of digits data
print(digits.data.shape)
#check the shape of digits target
print(digits.target.shape)

(1797, 64)
(1797,)


In [76]:
#let us use the linear regression used in the previous lab
N = digits.data.shape[0] #Number of data points
n = digits.data.shape[1] #Dimension of data points
A = digits.data

In [77]:
np.shape(A)

(1797, 64)

In [78]:
#In the following code, we create a Nx1 vector of target labels
y = 1.0*np.ones([A.shape[0],1])
for i in range(digits.target.shape[0]):
  y[i] = digits.target[i]

$\textbf{Direct OLSLR}$

In [79]:
#Now we will define a function which will compute and return the function value 
def evalf1(x, n):  
  #Input: x is a numpy array of size n 
  assert type(x) is np.ndarray  #do not allow arbitrary type arguments 
  assert len(x) == n #do not allow arbitrary size arguments 
  #after checking if the argument is valid, we can compute the objective function value
  #compute the function value and return it 
  fval = np.linalg.norm(np.matmul(A,x) - y)
  fval = 0.5 * (fval)**2
  return (fval)

In [80]:
def evalg1(x, n):
  assert type(x) is np.ndarray 
  assert len(x) == n
  return np.matmul(A.T, np.matmul(A, x) - y)


In [81]:
def evalh1(x,n):
  assert type(x) is np.ndarray  #do not allow arbitrary type arguments 
  assert len(x) == n #do not allow arbitrary size arguments 
  return np.matmul(A.T,A)

In [82]:
BACKTRACKING_LINE_SEARCH = 2

In [83]:
def compute_D_k1(x,n):
  assert type(x) is np.ndarray
  assert len(x) == n
  hess = evalh1(x,n)
  return np.linalg.inv(hess)

In [84]:
def compute_steplength_backtracking_scaled_direction1(x,n, gradf, direction, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(x) == n
  assert type(gradf) is np.ndarray and len(gradf) == n
  #assert type(direction) is np.ndarray and len(direction) == 2  
   
  #assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  
  
  #Complete the code 
  alpha = alpha_start
  gradf = evalg1(x,n)
  p=direction
  #np.matmul(np.matrix.transpose(gradf), p)
  while evalf1(x+alpha*p,n) > evalf1(x,n) + gamma*alpha*np.matmul(gradf.T, p) :
    alpha = rho*alpha
  return alpha

In [85]:
import math
def find_minimizer_Newtonmethod1(start_x, n, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments 
  assert len(start_x) == n #do not allow arbitrary size arguments 
  assert type(tol) is float and tol>=0.0
  
  x = start_x
  g_x = evalg1(x,n)
  h_x = evalh1(x,n)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    #implement the Newton's method here
    D_k=np.linalg.inv(evalh1(x,n))
    direction = np.matmul(D_k,-g_x)
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction1(x,n,g_x, direction, alpha_start, rho, gamma)  
    else:  
      raise ValueError('Line search type unknown. Please check!')
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x)))
    k += 1 #increment iteration
    g_x = evalg1(x, n) #compute gradient at new point
    
  return x, evalf1(x,n), k

In [86]:
def find_minimizer_BFGS_method_Newton1(start_x, n, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments 
  assert len(start_x) == n #do not allow arbitrary size arguments 
  assert type(tol) is float and tol>=0 
  
  x = start_x
  x0 = x
  g_x = evalg1(x,n)
  g0 = g_x

  
  if line_search_type == BACKTRACKING_LINE_SEARCH:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    #implement the Newton's method here
    
    x0 = x
    g_x = evalg1(x,n)
    g0 = g_x

    if k==0:
      B_k=np.identity(n)
    else:

      I = np.identity(n)
      
      mu_k = 1/np.matmul(np.transpose(y_k),s_k)
      B_k = np.add(np.matmul(np.matmul(np.subtract(I, mu_k*np.outer( s_k, np.transpose(y_k))),B_k), np.subtract(I, mu_k*np.outer(y_k,np.transpose(s_k)))), mu_k*np.outer( s_k, np.transpose(s_k)))
    direction = np.matmul(B_k,-g_x)
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction1(x,n,g_x, direction, alpha_start, rho, gamma)     
    else:  
      raise ValueError('Line search type unknown. Please check!')
    x = np.subtract(x, np.multiply(step_length,np.matmul(B_k, g_x)))
    g_x = evalg1(x, n)
    s_k = x-x0 
    y_k=  g_x-g0
    k += 1 #increment iteration
    g_x = evalg1(x, n) #compute gradient at new point
   
  return x, evalf1(x,n), k

$\textbf{Regularized OLSLR}$

In [87]:
#Now we will define a function which will compute and return the function value 
def evalf2(x, n, l ):  
  #Input: x is a numpy array of size n 
  assert type(x) is np.ndarray  #do not allow arbitrary type arguments 
  assert len(x) == n #do not allow arbitrary size arguments 
  #after checking if the argument is valid, we can compute the objective function value
  #compute the function value and return it 
  fval = np.linalg.norm(np.matmul(A,x) - y)
  fval = l/2*np.matmul(x.T,x) + 0.5 * (fval)**2
  return (fval)

In [88]:
def evalg2(x, n, l):
  assert type(x) is np.ndarray
  assert len(x) == n
  grad = l*x + np.matmul(A.T, np.matmul(A, x) - y)
  return grad

In [89]:
def evalh2(x,n,l):
  assert type(x) is np.ndarray  #do not allow arbitrary type arguments 
  assert len(x) == n #do not allow arbitrary size arguments 
  return np.matmul(A.T,A) + l*np.identity(n)

In [90]:
#line search type 
BACKTRACKING_LINE_SEARCH = 2

In [91]:
def compute_D_k2(x,n):
  assert type(x) is np.ndarray
  assert len(x) == n
  hess = evalh2(x,n)
  return np.linalg.inv(hess)

In [92]:
def compute_steplength_backtracking_scaled_direction2(x,n,l, gradf, direction, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(x) == n
  assert type(gradf) is np.ndarray and len(gradf) == n
  #assert type(direction) is np.ndarray and len(direction) == 2  
   
  #assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  
  
  #Complete the code 
  alpha = alpha_start
  gradf = evalg2(x,n,l)
  p=direction
  #np.matmul(np.matrix.transpose(gradf), p)
  while evalf2(x+alpha*p,n,l) > evalf2(x,n,l) + gamma*alpha*np.matmul(np.matrix.transpose(gradf), p) :
    alpha = rho*alpha
  
  return alpha

In [93]:
import math
def find_minimizer_Newtonmethod_Regularized2(start_x, n,l, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments 
  assert len(start_x) == n #do not allow arbitrary size arguments 
  assert type(tol) is float and tol>=0 
  
  x = start_x
  g_x = evalg2(x,n,l)
  h_x = evalh2(x,n,l)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    #implement the Newton's method here
    D_k=np.linalg.inv(evalh2(x,n,l))
    direction = np.matmul(D_k,-g_x)
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction2(x,n,l,g_x, direction, alpha_start, rho, gamma)  
    else:  
      raise ValueError('Line search type unknown. Please check!')
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x)))
    k += 1 #increment iteration
    g_x = evalg2(x, n,l) #compute gradient at new point
    
  return x, evalf2(x,n,l), k

In [94]:
def find_minimizer_BFGS_method2(start_x, n,l, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments 
  assert len(start_x) == n #do not allow arbitrary size arguments 
  assert type(tol) is float and tol>=0 
  
  x = start_x
  x0 = x
  g_x = evalg2(x,n,l)
  g0 = g_x

  if line_search_type == BACKTRACKING_LINE_SEARCH:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    #implement the Newton's method here
    
    x0 = x
    g_x = evalg2(x,n,l)
    g0 = g_x

    if k==0:
      B_k=np.identity(n)
    else:

      I = np.identity(n)
      
      mu_k = 1/np.matmul(np.transpose(y_k),s_k)

      B_k = np.add(np.matmul(np.matmul(np.subtract(I, mu_k*np.outer( s_k, np.transpose(y_k))),B_k), np.subtract(I, mu_k*np.outer(y_k,np.transpose(s_k)))), mu_k*np.outer( s_k, np.transpose(s_k)))
    direction = np.matmul(B_k,-g_x)
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction2(x,n,l,g_x, direction, alpha_start, rho, gamma)     
    else:  
      raise ValueError('Line search type unknown. Please check!')
    x = np.subtract(x, np.multiply(step_length,np.matmul(B_k, g_x)))
    g_x = evalg2(x, n,l)
    s_k = x-x0 
    y_k=  g_x-g0
    k += 1 #increment iteration
   
  return x, evalf2(x,n,l), k

#Que1.

$ \textbf{-Backtracking line Search(Newton Method) by Direct OLSLR Method}:$

In [95]:
n = 64
alpha = 0.99
rho = 0.5
gamma = 0.5
my_tol= 1e-3
my_start_x =np.zeros((64, 1))

x, opt_fval, num_iters = find_minimizer_Newtonmethod1(my_start_x, n, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print("x_star=", x)

LinAlgError: ignored

$ \textbf{Backtracking line Search(Newton Method) by Regularized OLSLR Method}:$

In [96]:
n = 64
alpha = 0.99
rho = 0.5
gamma = 0.5
my_tol= 1e-3
l=0.001
my_start_x =np.zeros((n, 1))

x, opt_fval, num_iters = find_minimizer_Newtonmethod_Regularized2(my_start_x, n,l, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print("x_star_f_lambda=", x)

x_star_f_lambda= [[ 0.00000000e+00]
 [ 9.69076882e-02]
 [-4.32192762e-03]
 [-7.75916338e-03]
 [ 7.49591987e-02]
 [ 1.13946582e-02]
 [-2.71293921e-02]
 [-7.34410664e-03]
 [ 9.98267907e-01]
 [-2.88089376e-02]
 [ 1.18688356e-01]
 [ 6.60922719e-02]
 [-5.57075915e-02]
 [-6.97056149e-02]
 [ 9.65844014e-02]
 [ 2.55196013e-01]
 [-7.29830665e-01]
 [ 2.42711745e-02]
 [ 7.73241544e-02]
 [-2.33008498e-02]
 [-5.64077619e-02]
 [ 5.72413971e-02]
 [-4.88675248e-02]
 [-2.62555998e-01]
 [-9.06071169e-01]
 [-1.49769638e-01]
 [ 5.64022790e-02]
 [ 8.96665319e-02]
 [ 8.39315938e-02]
 [ 9.85410047e-02]
 [ 1.69269848e-03]
 [-2.96649688e+00]
 [ 0.00000000e+00]
 [-1.54361470e-01]
 [-9.32404639e-03]
 [ 1.39497978e-01]
 [-3.69237437e-02]
 [ 5.46111773e-02]
 [-9.20425586e-03]
 [ 0.00000000e+00]
 [ 1.03326506e-01]
 [ 1.23983484e-01]
 [-1.37635230e-02]
 [ 5.40029021e-03]
 [ 1.31185700e-01]
 [ 5.49577815e-02]
 [ 2.24935899e-02]
 [ 7.48046265e-03]
 [ 6.17507773e-01]
 [ 2.44100619e-02]
 [ 1.42356436e-03]
 [-6.21114801e

$\textbf{Explain observations.}$

***We are not able to find the minimum using Direct OLSLR.We are able to find the minimum using regularized OLSLR. because in between hessian becomes non-invertible (singular matrix) and we run into trouble. Reguralized OLSLR makes hessian positive definite by adding lambda times identity and hence we are able to find the hessian inverse and then we find the minimum.***

#Que2.

$ \textbf{Backtracking line Search(BFGS method) by Direct OLSLR Method}:$

In [97]:
alpha = 0.99
rho = 0.5
gamma = 0.5
my_tol= 1e-3
my_start_x =np.zeros((n, 1))
x, opt_fval, num_iters= find_minimizer_BFGS_method_Newton1(my_start_x,n,my_tol,BACKTRACKING_LINE_SEARCH,0.99, 0.5,0.5)
print('Backtracking line Search(BFGS Method) by Direct OLSLR Method')
print("x_star_f=", x)

Backtracking line Search(BFGS Method) by Direct OLSLR Method
x_star_f= [[ 0.00000000e+00]
 [ 9.69033595e-02]
 [-4.32277206e-03]
 [-7.76028207e-03]
 [ 7.49594378e-02]
 [ 1.13947198e-02]
 [-2.71328215e-02]
 [-7.33177570e-03]
 [ 9.98338024e-01]
 [-2.88095544e-02]
 [ 1.18688288e-01]
 [ 6.60916271e-02]
 [-5.57069865e-02]
 [-6.97063699e-02]
 [ 9.65876410e-02]
 [ 2.55182267e-01]
 [-7.29828835e-01]
 [ 2.42709925e-02]
 [ 7.73249590e-02]
 [-2.33000285e-02]
 [-5.64086139e-02]
 [ 5.72426812e-02]
 [-4.88717646e-02]
 [-2.62467851e-01]
 [-9.06562366e-01]
 [-1.49767793e-01]
 [ 5.64019539e-02]
 [ 8.96663593e-02]
 [ 8.39318156e-02]
 [ 9.85411936e-02]
 [ 1.69317563e-03]
 [-2.96805612e+00]
 [ 0.00000000e+00]
 [-1.54362337e-01]
 [-9.32361244e-03]
 [ 1.39497628e-01]
 [-3.69234836e-02]
 [ 5.46111776e-02]
 [-9.20505021e-03]
 [ 0.00000000e+00]
 [ 1.03279508e-01]
 [ 1.23983259e-01]
 [-1.37639603e-02]
 [ 5.40087799e-03]
 [ 1.31185107e-01]
 [ 5.49570764e-02]
 [ 2.24938233e-02]
 [ 7.47978301e-03]
 [ 6.17754849e-01

$ \textbf{Backtracking line Search(BFGS Method) by Regularized OLSLR Method}:$

In [98]:
n = 64
alpha = 0.99
rho = 0.5
gamma = 0.5
my_tol= 1e-3
l=0.001
my_start_x =np.zeros((n, 1))

x, opt_fval, num_iters= find_minimizer_BFGS_method2(my_start_x,n,l,my_tol,BACKTRACKING_LINE_SEARCH,0.99, 0.5,0.5)
print('Backtracking line Search(BFGS Method) by Regularized OLSLR Method')
print("x_star_f_lambda=", x)

Backtracking line Search(BFGS Method) by Regularized OLSLR Method
x_star_f_lambda= [[ 0.00000000e+00]
 [ 9.69076908e-02]
 [-4.32192735e-03]
 [-7.75916227e-03]
 [ 7.49591985e-02]
 [ 1.13946582e-02]
 [-2.71293891e-02]
 [-7.34411886e-03]
 [ 9.98267966e-01]
 [-2.88089382e-02]
 [ 1.18688357e-01]
 [ 6.60922725e-02]
 [-5.57075918e-02]
 [-6.97056144e-02]
 [ 9.65843985e-02]
 [ 2.55196029e-01]
 [-7.29830898e-01]
 [ 2.42711754e-02]
 [ 7.73241538e-02]
 [-2.33008504e-02]
 [-5.64077614e-02]
 [ 5.72413961e-02]
 [-4.88675210e-02]
 [-2.62556084e-01]
 [-9.06070691e-01]
 [-1.49769640e-01]
 [ 5.64022791e-02]
 [ 8.96665322e-02]
 [ 8.39315936e-02]
 [ 9.85410047e-02]
 [ 1.69269799e-03]
 [-2.96649543e+00]
 [ 0.00000000e+00]
 [-1.54361470e-01]
 [-9.32404678e-03]
 [ 1.39497978e-01]
 [-3.69237437e-02]
 [ 5.46111773e-02]
 [-9.20425537e-03]
 [ 0.00000000e+00]
 [ 1.03326478e-01]
 [ 1.23983485e-01]
 [-1.37635227e-02]
 [ 5.40029006e-03]
 [ 1.31185700e-01]
 [ 5.49577820e-02]
 [ 2.24935895e-02]
 [ 7.48046661e-03]
 [ 6.

$\textbf{Explain observations.}$

***We are able to find the minimum using both the mathod regularized OLSLR and Direct OLSLR because in between hessian does not becomes non-invertible (singular matrix) and we do run into trouble because of approximation of hessian. Reguralized OLSLR makes approximated hessian positive definite by adding lambda times identity and hence we are able to find the hessian inverse and then we find the minimum.***